<a href="https://colab.research.google.com/github/SyaoranClone/Persona-Chatbot/blob/master/Persona_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [2]:
#install Apex
%%writefile setup.sh

git clone https://github.com/NVIDIA/apex
cd apex
pip install -v --no-cache-dir ./

Overwriting setup.sh


In [ ]:
!sh setup.sh

In [18]:
import json
import time
import os
from itertools import chain
from argparse import ArgumentParser
from collections import defaultdict
from tqdm import tqdm, trange

import torch
import numpy as np
from torch.nn.parallel import DistributedDataParallel
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, DistributedSampler, SequentialSampler
from transformers import (AdamW, OpenAIGPTDoubleHeadsModel, OpenAIGPTTokenizer,
                                 GPT2DoubleHeadsModel, GPT2Tokenizer, WEIGHTS_NAME, CONFIG_NAME)
from transformers import get_linear_schedule_with_warmup

In [ ]:
torch.cuda.device_count()

1

In [5]:
#According to Huggingface Convai tutorial
SPECIAL_TOKENS = ["<bos>", "<eos>", "<speaker1>", "<speaker2>", "<pad>"]
ATTR_TO_SPECIAL_TOKEN = {'bos_token': '<bos>', 'eos_token': '<eos>', 'pad_token': '<pad>',
                         'additional_special_tokens': ['<speaker1>', '<speaker2>']}
MODEL_INPUTS = ["input_ids", "mc_token_ids", "lm_labels", "mc_labels", "token_type_ids"]
PADDED_INPUTS = ["input_ids", "lm_labels", "token_type_ids"]

In [41]:
#define hyperparameters
class args:
  model_checkpoint = 'gpt2'
  device  = 'cuda' if torch.cuda.is_available() else 'cpu'
  lr  = 6.25e-5
  num_candidates = 2
  num_history  = 2 #Number of previous exchanges to keep in history
  fp16_training = "O1" #Set to O0, O1, O2 or O3 for fp16 training
  train_batch_size = 4
  valid_batch_size = 4
  num_epochs= 3
  lm_coef = 2.0
  mc_coef = 1.0
  max_norm= 1.0
  num_gpu = torch.cuda.device_count() #1
  gradient_accumulation_steps= 8
  local_rank= -1 # for distributed training
  url = "https://s3.amazonaws.com/datasets.huggingface.co/personachat/personachat_self_original.json"
  local_path= "/content/drive/My Drive/Colab Notebooks/Dataset/personachat_self_original.json"
  dataset_cache_path = "/content/drive/My Drive/Colab Notebooks/Dataset/dataset_cache/persona_cache.bin_GPT2Tokenizer"
  saved_model_dir = "/content/drive/My Drive/Colab Notebooks/Trained Models/persona_chatbot/"

args = args

In [7]:
class FacebookPersonaDataset():
  """
    Concatenate context segments in a single sequence [[bos+persona], [history], [reply+eos]]
    Tokenize and convert them to tensor
    
  """
  def __init__(self,url,cache_path,tokenizer = ''):
    self.file_path = url
    self.tokenizer = tokenizer
    self.cache_path = cache_path

  def _load_dataset(self):
    #self.cache_path = self.cache_path + '_' + type(self.tokenizer).__name__  # To avoid using GPT cache for GPT-2 and vice-versa
    if self.cache_path and os.path.isfile(self.cache_path):
      #load tokenized dataset
      dataset = torch.load(self.cache_path)
      print("dataset loaded")
    else:
      with open(self.file_path, "r", encoding="utf-8") as f:
        dataset = json.loads(f.read())
      
      def tokenize(obj):
        if isinstance(obj,str):
          return self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(obj))
        if isinstance(obj,dict):
          return dict((n, tokenize(o)) for n, o in obj.items())
        return list(tokenize(o) for o in obj)
      dataset = tokenize(dataset)
      torch.save(dataset,self.cache_path)
    return dataset

  def _pad_dataset(self,dataset, padding=0):
    """ Pad the dataset. This could be optimized by defining a Dataset class and padding at the batch level, but this is simpler. """
    max_l = max(len(x) for x in dataset["input_ids"])
    for name in PADDED_INPUTS:
        dataset[name] = [x + [padding if name != "lm_labels" else -100] * (max_l - len(x)) for x in dataset[name]]
    return dataset

  def _build_input(self,persona,history,reply,lm_labels = False,with_eos = True):
    """ Build a sequence of input from 3 segments: persona, history and last reply. """
    bos, eos, speaker1, speaker2 = self.tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[:-1])
    #bos, eos, speaker1, speaker2 = "<bos>", "<eos>", "<speaker1>", "<speaker2>"
    #sequence = [[bos] + list(persona) + ["his"] + history + ["rep"] + [reply]]
    sequence = [[bos] + list(chain(*persona))] + history + [reply + ([eos] if with_eos else [])] #add bos, eos
    sequence = [sequence[0]] + [[speaker2 if (len(sequence)-i) % 2 else speaker1] + s for i, s in enumerate(sequence[1:])] #add speaker1, speaker2
    #after concat: [[bos+persona], [history], [reply+eos]]
    instance = {}
    instance["input_ids"] = list(chain(*sequence))
    #segment
    instance["token_type_ids"] = [speaker2 if i % 2 else speaker1 for i, s in enumerate(sequence) for _ in s]
    #position
    instance["mc_token_ids"] = len(instance["input_ids"]) - 1
    #language model labels is used to calculate lm_loss
    instance["lm_labels"] = [-100] * len(instance["input_ids"]) #labels set to -100 are ignored (masked)
    if lm_labels:
        instance["lm_labels"] = ([-100] * sum(len(s) for s in sequence[:-1])) + [-100] + sequence[-1][1:]
    return instance

  def get_data_loaders(self):
    personachat_dataset = self._load_dataset()
    datasets = {"train": defaultdict(list), "valid": defaultdict(list)}
    for name,dataset in personachat_dataset.items():
      num_candidates = len(dataset[0]["utterances"][0]["candidates"]) #num_candidates are same for all dialoges
      if args.num_candidates > 0 and name == 'train':
        num_candidates =  min(num_candidates ,args.num_candidates)
      for dialoge in dataset:
        persona = dialoge["personality"].copy()
        for utterance in dialoge["utterances"]:
          history = utterance["history"][-(2*args.num_history+1):]
          for j, candidate in enumerate(utterance["candidates"][-num_candidates:]):
            #The last candidate is a ground truth reponse.
            lm_labels = bool(j==num_candidates-1)
            instance = self._build_input(persona,history,candidate,lm_labels)
            for input_name,data in instance.items():
              datasets[name][input_name].append(data) #datasets['train']['input_ids'] of [[c1 in u1],[c2 in u1],..,[c1 in u 7][c2 in u7]]
          datasets[name]["mc_labels"].append(num_candidates - 1) #
          datasets[name]["n_candidates"] = num_candidates

    #pad input and convert to tensor
    print("pad input and convert to tensor")
    tensor_datasets = {"train": [], "valid": []}
    for dataset_name, dataset in datasets.items():
      dataset =  self._pad_dataset(dataset, padding=self.tokenizer.convert_tokens_to_ids(SPECIAL_TOKENS[-1]))
      for input_name in MODEL_INPUTS:
        tensor =  torch.tensor(dataset[input_name])
        if input_name != "mc_labels":
          tensor = tensor.view((-1, datasets[dataset_name]["n_candidates"]) + tensor.shape[1:])
        tensor_datasets[dataset_name].append(tensor)

    train_dataset, valid_dataset = TensorDataset(*tensor_datasets["train"]), TensorDataset(*tensor_datasets["valid"])
    return train_dataset, valid_dataset 

In [12]:
persona_dataset = FacebookPersonaDataset(args.local_path,args.dataset_cache_path,tokenizer)

In [13]:
train_dataset, valid_dataset = persona_dataset.get_data_loaders() 

dataset loaded
pad input and convert to tensor


In [14]:
train_dataset.tensors[0].shape

torch.Size([131438, 2, 280])

In [9]:
def add_special_token(model,tokenizer):
  origin_num_tokens = len(tokenizer.encoder)
  num_special_tokens = tokenizer.add_special_tokens(ATTR_TO_SPECIAL_TOKEN)
  if num_special_tokens > 0:
        model.resize_token_embeddings(new_num_tokens=origin_num_tokens + num_special_tokens)

In [ ]:
model = GPT2DoubleHeadsModel.from_pretrained(args.model_checkpoint)
tokenizer = GPT2Tokenizer.from_pretrained(args.model_checkpoint)
model.to(args.device)

In [11]:
add_special_token(model,tokenizer)

In [34]:
def train(train_dataset,valid_dataset,model,tokenizer):
  train_sampler = RandomSampler(train_dataset) if args.local_rank== -1 else DistributedSampler(train_dataset)
  train_loader = DataLoader(train_dataset,batch_size=args.train_batch_size,sampler=train_sampler)
  optimizer = AdamW(model.parameters(),lr = args.lr,correct_bias=True)

  if args.fp16_training:
    from apex import amp
    # Allow Amp to perform casts as required by the opt_level 
    model,optimizer = amp.initialize(model,optimizer,opt_level=args.fp16_training)
  # Linearly decrease the learning rate from lr to zero
  #scheduler = PiecewiseLinear(optimizer, "lr", [(0, args.lr), (args.num_epochs* len(train_loader), 0.0)])
  scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps = 0, # Default value in run_glue.py
                                                num_training_steps = args.num_epochs+1)
  tr_loss, loss = 0.0, 0.0
  global_step = 0
  metrics = {
      "nll" : 1000.0,
      "accurancy": 0.0
  }
  if args.num_gpu > 1:
    model = torch.nn.DataParallel(model)

  if args.local_rank != -1:
    model = DistributedDataParallel(model,device_ids = [args.local_rank],output_device=args.local_rank,find_unused_parameters=True)
  for _ in range(args.num_epochs):
    for step,batch in enumerate(tqdm(train_loader,disable= args.local_rank not in [-1,0])):
      model.train()
      batch = tuple(input_tensor.to(args.device) for input_tensor in batch )
      input_ids, mc_token_ids, lm_labels, mc_labels, token_type_ids = batch

      (lm_loss),(mc_loss), *_ = model(input_ids,token_type_ids=token_type_ids,mc_labels=mc_labels,lm_labels=lm_labels)

      loss = (lm_loss * args.lm_coef + mc_loss * args.mc_coef)/args.gradient_accumulation_steps
      if args.fp16_training:
        with amp.scale_loss(loss, optimizer) as scaled_loss:
          scaled_loss.backward()
        torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_norm)
      else:
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0. This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(),args.max_norm)

      if args.num_gpu > 1:
        # mean() to average on multi-gpu parallel training
        loss = loss.mean()
      
      if step % 100 == 0:
        print("Loss for step {} is {}".format(step, loss))

      tr_loss += loss.item()
      global_step += 1
      if (step+1) % args.gradient_accumulation_steps == 0:
        optimizer.step()
        # Update the learning rate.
        scheduler.step()
        optimizer.zero_grad()
        if args.local_rank in [-1,0]:
          metrics = evaluate(model, valid_dataset, metrics, tokenizer)
        
  return tr_loss/global_step,metrics


In [35]:
def evaluate(model,valid_dataset,metrics,tokenizer):
  valid_sample = SequentialSampler(valid_dataset) if  args.local_rank== -1 else DistributedSampler(train_dataset)
  eval_dataloader = DataLoader(valid_dataset,sampler=valid_sample,batch_size=args.valid_batch_size)

  print(' * Running Evaluation')
  print(' * Num of examples: %d" ',len(valid_dataset))
  print(' * Batch size: %d" ',args.valid_batch_size)
  nlls = None
  accs = None
  for step,batch in enumerate(tqdm(eval_dataloader,desc="Evaluating")):
    model.eval()
    with torch.no_grad():
      batch = tuple(input_tensor.to(args.device) for input_tensor in batch)
      input_ids, mc_token_ids, lm_labels, mc_labels, token_type_ids = batch
      lm_logits, mc_logits, *_ = model(
          input_ids, token_type_ids=token_type_ids, mc_token_ids=mc_token_ids)
      
      lm_logits_flat_shifted = lm_logits[..., :-1, :].contiguous().view(-1, lm_logits.size(-1))
      lm_labels_flat_shifted = lm_labels[..., 1:].contiguous().view(-1)
            
      x = ((lm_logits_flat_shifted, mc_logits), (lm_labels_flat_shifted, mc_labels))
      #convert pytorch tensor to numpy array to calculate
      nll = torch.nn.CrossEntropyLoss(ignore_index=-100)(x[0][0], x[1][0]).detach().cpu().numpy()
      acc = torch.sum((torch.max(x[0][1], 1)[1] == x[1][1]).int()).detach().cpu().numpy().mean()

    if nlls is None:
      nlls = nll
      accs = acc
    else:
      nlls = np.append(nlls, nll)
      accs = np.append(accs, acc)
  
  nlls_mean = np.mean(nlls)
  accs_mean = np.mean(accs)
    
  if accs_mean>metrics['accuracy'] and nlls_mean<metrics['nll']:
    print(" * New high accuracy and nll! {} {} ".format(accs_mean, nlls_mean))
    metrics.update({'nll': nlls_mean, 'accuracy': accs_mean})
    #Save model if getting high accuracy
    model_to_save = model.module if hasattr(model, "module") else model #Take care of distributed/parallel training
    model_to_save.save_pretrained(args.output_dir)
    tokenizer.save_pretrained(args.output_dir)
    
  return metrics 

In [44]:
train(train_dataset,valid_dataset,model,tokenizer)

  0%|          | 0/32860 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/transformers/modeling_gpt2.py:892: FutureWarning: The `lm_labels` argument is deprecated and will be removed in a future version, use `labels` instead.
  FutureWarning,
  0%|          | 0/32860 [00:00<?, ?it/s]

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


RuntimeError: ignored

In [ ]:
x = (([1,2,3],[[3,4,3],[5,8,5],[6,6,11]]),([3,4,3],[5,5,5]))

In [43]:
torch.cuda.empty_cache()

In [ ]:
torch.sum((torch.max(torch.tensor(x[0][1]), 1)[1] == torch.tensor(x[1][1])).int()).detach().cpu().numpy().mean()

0.0